In [31]:
import numpy as np
import matplotlib.pyplot as plt 


def PoroFracNewton(b, kz, bj, aj, xn):
    global N, Kc, alpha, z  # Assuming these globals are defined elsewhere

    #delta = 1- 3.2/4.9 # delta = vt/c, should be an input to this function.
    
    
    bn = b / N

    # Integration of y = Mathcal(I)
    y = np.zeros(N + 1)
    for i in range(1, N + 1):
        y[i] = bn * np.sum(kz[i, :] * aj * bj * xn)

    y2 = y**(-2 * alpha - 1)

    # r1 calculation
    r1 = np.zeros(N + 1)
    r1[1:] = 1.0 / (z[1:]**alpha * (z[1:]**(2 - alpha) + 1))

    # Residues calculation
    Res = np.zeros(N + 1)
    Res[0] = xn[0] - (2 * Kc)**(-2 * alpha)
    Res[1:] = xn[1:] * r1[1:]  - (y[1:]**(-2 * alpha) - 1)
    Res[N] = 2 * bn * np.sum(bj * aj * xn) + Kc

    # Jacobian calculation
    jcb = np.zeros((N + 1, N + 1))  # Create a square Jacobian matrix
    for i in range(1, N + 1):
        jcb[i, :] = 2 * alpha * bn * y2[i] * kz[i, :] * bj * aj
        jcb[i, i] += r1[i]

    jcb[-1, :] = 2 * bn * bj * aj
    jcb[0, 0] = 1

    dx = np.linalg.solve(jcb, Res.T)  # Solve the linear system 

    return y, Res, jcb, dx

In [32]:

def PoroFracNonlinear(x0, itmax, xtol):
    global Kc, alpha, N, drmax, z, zp  # Assuming these globals are defined

    # Set grids
    rmax = 1 - drmax
    theta = np.linspace(0, np.pi/2 * rmax, N + 1)

    zp = np.tan(theta)
    z = zp**2

    # Quadrature parameters
    bat = 0.5
    bbt = np.ones(N + 1)
    bbt[1:-1] = 2  

    bas = 1/3
    bbs = np.ones(N + 1)
    bbs[1::2] = 4
    bbs[2::2] = 2

    # Calculate A(theta)
    sint = np.sin(theta)
    cost = np.cos(theta)
    a2 = 2 * alpha
    a22 = a2 - 2

    A = cost**a22 / (sint**2 * cost**a22 + cost**2 * sint**a22)

    # Calculate matrix kz
    kz = np.zeros((N + 1, N + 1))
    for i in range(N+1):
        kz[i, :] = (
            (z[i] / zp - zp) * np.log(np.abs((zp + zp[i]) / (zp - zp[i]))) - 2 * zp[i]
        )

    np.fill_diagonal(kz, -2 * zp)  # Efficient way to set the diagonal
    kz[:, 0] = 0
    kz[:, -1] = -4 * zp

    # Newton's iteration
    ba = bas * rmax
    bb = bbs
    xn = x0

    mres = np.zeros(itmax)
    imres = np.zeros(itmax)
    mdx = np.zeros(itmax)

    for it in range(itmax):
        y, Res, jcb, dx = PoroFracNewton(ba, kz, bb, A, xn)  # Call your other function

        mres[it], imres[it] = np.max(np.abs(Res)), np.argmax(np.abs(Res))
        mdx[it] = np.max(np.abs(dx))

        xn1 = xn - dx.T  # Transpose for correct dimensions
        xn = xn1

        if mdx[it] < xtol:
            break

    if mdx[it] > xtol:
        print(f"Not converged, max(dx) ({mdx[it]:.3e}) > tol ({xtol:.3e})")

    return y, xn, dx 


In [33]:
def PoroFrac(KcA, alA):
    global Kc, alpha, N, drmax, z  # Assuming these globals are needed 

    ik = len(KcA)
    ia = len(alA)

    N = 800
    drmax = 1/10
    xtol = 1e-4
    maxnew = 20

    x0 = -np.ones(N + 1)  # Initial guess

    ha = np.zeros((ia, ik, N + 1))
    za = np.zeros((ia, ik, N + 1))

    # Main loop
    for j in range(ia):
        alpha = alA[j]
        if j > 0:
            x0 = xn0.copy() 

        for i in range(ik):
            Kc = KcA[i]
            ha[j, i, :], xn, dx = PoroFracNonlinear(x0.copy(), maxnew, xtol) 
            x0 = xn.copy()
            za[j, i, :] = z  # Assuming 'z' is calculated within PoroFracNonlinear 
            if i == 0:
                xn0 = xn.copy()

    return za, ha